In [1]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from typing import List
import scipy.stats as stats
from sklearn.linear_model import LinearRegression

data_dir = Path(os.getcwd()).parent.parent.parent / 'Datasets'
data_path = data_dir / 'car-price-predicition.csv'

In [2]:
df = pd.read_csv(data_path)
response_variable = 'selling_price'
df = df.rename(columns={response_variable : 'y'})
features, target = df.drop('y', axis=1), df['y']

In [17]:
# Expectation -> Dataset is already clean
# # Feature transformation
# # Missing value handling / Sanitization

# Only check:
# Is response variable numeric? -> not? throw error
# Multi-colinearity

In [3]:
# Parameters for dataset
n_samples = 100  # Number of samples
n_features = 3   # Number of features

# Set a random seed for reproducibility
np.random.seed(42)

# Generate random feature data
X = np.random.rand(n_samples, n_features) * 10  # Scale feature values

# Define coefficients for the linear relationship
true_coefficients = np.array([3.5, -2.7, 2.0])

# Generate the target variable with some noise
noise = np.random.randn(n_samples) * 2  # Add noise to the data
y = X.dot(true_coefficients) + noise  # Linear relationship with noise

# Convert to DataFrame for readability
data = pd.DataFrame(X, columns=[f"feature_{i+1}" for i in range(n_features)])
data['y'] = y

features, target = data.drop('y', axis=1), data['y']

num_rows = 10
features = features[0:num_rows]
target = target[0:num_rows]

In [6]:
def linear_regression(features: np.array, target: np.array) -> List:
    lr = LinearRegression()
    lr.fit(features, target)

    return [lr.intercept_] + list(lr.coef_)

In [7]:
linear_regression(features, target)

[1.3497195647696483, 2.8886754379938293, -2.706440097788314, 2.280006450155889]

In [38]:
class LinearRegressionNumpy:
    def __init__(self, significance_level: float = 0.05):
        self.significance_level = significance_level

    def _is_degerate_matrix(self) -> int:
        # is full or low
        rank = np.linalg.matrix_rank(self.X)
        self.full_rank = True if rank == min(self.rows, self.cols) else False
        self.low_rank = True if self.rows < self.cols else False
        
        self.degerate = self.full_rank and not self.low_rank
        
    def _add_constant(self) -> None:
        self.X = np.column_stack((np.ones(self.rows), self.X))
    
    def _standarization(self):
        self.X = (self.X - np.mean(self.X, axis=0)) / np.std(self.X, axis=0)

    def closed_form_solution(self, X, y):
        self.covar_inv = np.linalg.inv(np.dot(X.T, X))
        return np.dot(self.covar_inv, np.dot(X.T, y))

    def fit(self, X: pd.DataFrame, y: pd.Series) -> None:
        self.feature_names = X.columns
        self.X, self.y = X.to_numpy(), y.to_numpy()
        self.rows, self.cols = X.shape
    
        self.ddof_ssr = self.cols
        self.ddof_sse = self.rows - self.cols - 1
        self.ddof_sst = self.rows - 1

        self.X_bar = np.mean(self.X, axis=0)
        self.y_bar = np.mean(y)
        self.t_alpha_2 = stats.t.ppf(self.significance_level, self.ddof_sse)

        self._add_constant()

        if not self._is_degerate_matrix():
            self.coefficients = self.closed_form_solution(self.X, self.y)
        else:
            raise("Degerate Data Matrix! Cannot fit Linear Model")
        self.y_hat = self.predict(self.X)
        
        self.compute_ssr()
        self.compute_sse()
        self.compute_sst()
        self.compute_s2()
        self.compute_r2()
        self.compute_coefficient_std_errors()
        self.coefficient_t_statistic()
        self.coefficient_t_stat_pvalue()
        self.compute_model_f_stat()
        self.coefficient_confidence_interval()

    def compute_ssr(self):
        self.ssr = np.sum([(y_hat_i - self.y_bar)**2 for y_hat_i in self.y_hat])
        self.s1_2 = self.ssr / self.ddof_ssr
    def compute_sse(self):
        self.sse = np.sum([(y_hat_i - y_i)**2 for y_hat_i, y_i in zip(self.y_hat, self.y)])
    def compute_sst(self):
        self.sst = self.ssr + self.sse
    
    def compute_s2(self):
        self.s2 = self.sse / self.ddof_sse
        self.s = np.sqrt(self.s2)
    
    def compute_r2(self):
        self.r2 =  1 - (self.sse/self.sst)
        self.r2_adj = 1 - ((self.sse/self.ddof_sse)/(self.sst/self.ddof_sst))

    def compute_coefficient_std_errors(self) -> List:
        self.coeff_std_errors = [self.s * np.sqrt(self.covar_inv[idx][idx]) for idx in range(self.cols + 1)]
    def coefficient_t_statistic(self) -> List:
        self.coeff_t_vals = [coeff/std_err for coeff, std_err in zip(self.coefficients, self.coeff_std_errors)]
    def coefficient_t_stat_pvalue(self):
        self.coeff_p_vals = [1-stats.t.cdf(t_val, self.ddof_sse) for t_val in self.coeff_t_vals]
    
    def compute_model_f_stat(self):
        self.f_stat = self.s1_2/self.s2
        self.f_pval = 1 - stats.f.cdf(self.f_stat, self.ddof_ssr, self.ddof_sse)
    def coefficient_confidence_interval(self):
        self.coeff_ci = [(coeff-(self.t_alpha_2*std_err), coeff+(self.t_alpha_2*std_err)) for coeff, std_err in zip(self.coefficients, self.coeff_std_errors)]

    def predict(self, X: np.array):
        return np.dot(X, self.coefficients)
    
    def summary(self):
        # Format the output similar to statsmodels' OLS summary
        print("OLS Regression Results")
        print("="*80)
        print(f"Dep. Variable:                   y    R-squared:             {self.r2:.3f}")
        print(f"Model:                         OLS    Adj. R-squared:        {self.r2_adj:.3f}")
        print(f"Method:             Least Squares   F-statistic:             {self.f_stat:.2f}")
        print(f"Date:             {pd.Timestamp.now():%a, %d %b %Y}  Prob (F-statistic):     {self.f_pval:.2e}")
        print(f"Time:                  {pd.Timestamp.now():%H:%M:%S}")
        print("\nNo. Observations:                {0}    Df Residuals:             {1}".format(self.rows, self.ddof_sse))
        print(f"Df Model:                         {self.ddof_ssr}\n")

        # Coefficients Table
        print(f"{'':<10}{'coef':>10} {'std err':>10} {'t':>10} {'P>|t|':>10} {'[0.025':>10} {'0.975]':>10}")
        print("-"*80)
        for i, (coeff, std_err, t_val, p_val, ci) in enumerate(zip(
            self.coefficients, self.coeff_std_errors, self.coeff_t_vals, self.coeff_p_vals, self.coeff_ci)):
            feature = "const" if i == 0 else self.feature_names[i-1]
            print(f"{feature:<10}{coeff:>10.4f} {std_err:>10.3f} {t_val:>10.3f} {p_val:>10.3f} {ci[0]:>10.3f} {ci[1]:>10.3f}")
        print("="*80)

    def get_params(self):
        params = {
            "R_squared": self.r2,
            "Adj_R_squared": self.r2_adj,
            "F_statistic": self.f_stat,
            "F_pvalue": self.f_pval,
            "Coefficients": {
                "Names": ["const"] + list(self.feature_names),
                "Values": self.coefficients,
                "Std_Errors": self.coeff_std_errors,
                "T_Values": self.coeff_t_vals,
                "P_Values": self.coeff_p_vals,
                "Confidence_Intervals": self.coeff_ci
            },
            "SSR": self.ssr,
            "SSE": self.sse,
            "SST": self.sst,
            "Residual_Std_Error": self.s,
        }
        return params

In [39]:
lr = LinearRegressionNumpy()
lr.fit(features, target)
lr.summary()

OLS Regression Results
Dep. Variable:                   y    R-squared:             0.993
Model:                         OLS    Adj. R-squared:        0.989
Method:             Least Squares   F-statistic:             279.55
Date:             Wed, 06 Nov 2024  Prob (F-statistic):     7.82e-07
Time:                  23:33:16

No. Observations:                10    Df Residuals:             6
Df Model:                         3

                coef    std err          t      P>|t|     [0.025     0.975]
--------------------------------------------------------------------------------
const         1.3497      1.980      0.682      0.260      5.198     -2.498
feature_1     2.8887      0.240     12.043      0.000      3.355      2.423
feature_2    -2.7064      0.181    -14.936      1.000     -2.354     -3.059
feature_3     2.2800      0.178     12.796      0.000      2.626      1.934


In [4]:
import statsmodels.api as sm

In [5]:
features_w_const = sm.add_constant(features)

In [7]:
model_1 = sm.OLS(target, features).fit()

In [8]:
model_1.summary()

/opt/anaconda3/envs/ds-env/lib/python3.10/site-packages/scipy/stats/_axis_nan_policy.py:418: UserWarning: `kurtosistest` p-value may be inaccurate with fewer than 20 observations; only n=10 observations were given.
  return hypotest_fun_in(*args, **kwds)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.995
Model:                            OLS   Adj. R-squared (uncentered):              0.993
Method:                 Least Squares   F-statistic:                              508.1
Date:                Fri, 08 Nov 2024   Prob (F-statistic):                    1.50e-08
Time:                        13:33:56   Log-Likelihood:                         -16.369
No. Observations:                  10   AIC:                                      38.74
Df Residuals:                       7   BIC:                                      39.65
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
feature_1      3.0207      0.136     22.216      0.000       2.699       3.342
feature_2     -2.6177      0.121    -21.601      0.000      -2.904      -2.331
feature_3      2.3491      0.141     16.686      0.000       2.016       2.682
==============================================================================
Omnibus:                        4.631   Durbin-Watson:                   1.668
Prob(Omnibus):                  0.099   Jarque-Bera (JB):                1.150
Skew:                          -0.006   Prob(JB):                        0.563
Kurtosis:                       1.339   Cond. No.                         2.88
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""